You can download all the neccesary models for this script here: https://drive.google.com/drive/folders/1aqpLhtbm3Yhs2eAMOiuhLTH2T1tsO0k3?usp=sharing. You have to extract everything from the downloaded zip. 

Use 'pip install -r requirements.txt' where requirements.txt is the path to the requirements file downloaded, to install all the packages needed to run this script.

In [ ]:
import tensorflow as tf
from resizeimage import resizeimage
from PIL import Image, ImageOps
import numpy as np
import cv2
import joblib
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
CLASS_NAMES_WITHOUT_DISGUST = ["Angry", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

In CNNModel1 you should put the path to the folder called 'CNNmodel3'.
In CNNModel2 you should put the path to the folder called 'CNNmodel6'.

In sgdModel you should put the path to the file called '39.74%_Smote_SGD.joblib'

In [ ]:
CNNModel1 = tf.keras.models.load_model("")
CNNModel2 = tf.keras.models.load_model("")

sgdModel = joblib.load("")

In [ ]:
def predict(image):
    rgb_image = cv2.cvtColor(np.float32(image), cv2.COLOR_GRAY2RGB)
    rgb_image = rgb_image.reshape(1, 48, 48, 3)
    image = image.reshape(1, 48, 48, 1)
    
    CNNModel1_predictions = CNNModel1.predict(image)
  
    CNNModel2_predictions = CNNModel2.predict(rgb_image) # This transfer learning model expects rgb input
    
    return CNNModel1_predictions, CNNModel2_predictions
    

def preprocess_image(image):
    image = resizeimage.resize_cover(image, [48,48])
    image = ImageOps.grayscale(image)
    image = np.array(image)
    image = image.astype("int32")
    image = image/255.0
    image = image.reshape(48, 48, 1)
    return image

In [ ]:
def import_test_sample(test_path):
    with open(test_path, 'r+b') as f:
        with Image.open(f) as image:
            test_image = resizeimage.resize_cover(image, [48,48])
    return test_image

In [ ]:
def predictCombined(predictions):
    y_pred = np.array([0,0,0,0,0,0])
    
    for prediction in predictions:
        y_pred = y_pred + np.array(prediction)
    return y_pred

In [ ]:
def click_and_crop(event, x, y, flags, param):
    global refPt, cropping
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

This function will get your webcam and open a window. You should drag a square to crop the window. This cropped window will be used to predict the emotion, it should only show your face. 

It will show the prediction in the output and the prediction distribution. 

In [ ]:
refPt = []
cam = cv2.VideoCapture(0) # Put here another number for another cam
if cam.isOpened():  # try to get the first frame
    print('Cam is found')
else:
    print("Cam is not found")

while True:
    ret_val, img = cam.read()
    if ret_val == False:
        cam.release()
        print("Cam is not working correctly")
    if cv2.waitKey(1) == 27: 
        break  # esc to quit
    if len(refPt) == 2:
        roi = img[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
        cv2.imshow("Cropped Webcam", roi)
        
        test_image = cv2.resize(roi, [48,48])
        test_image = Image.fromarray(test_image)
        test_image = ImageOps.grayscale(test_image)

        image1 = preprocess_image(test_image)
        CNNModel1_predictions, CNNModel2_predictions = predict(image1)

        predictions = [CNNModel1_predictions, CNNModel2_predictions]

        test_image = ImageOps.grayscale(test_image)

        test_image = np.array(test_image)
        test_image = test_image.astype("int32")
        test_image = test_image/255.0
        test_image = test_image.reshape(48, 48, 1)

        predictions.append(sgdModel.predict_proba(np.reshape(test_image, (1, 2304)))[0])
        
        y_pred = predictCombined(predictions)
        
        clear_output(wait=True)
        
        print("Predicted emotion:\n", CLASS_NAMES_WITHOUT_DISGUST[np.argmax(y_pred[0])])
        
        plt.bar(CLASS_NAMES_WITHOUT_DISGUST, y_pred[0])
        plt.show()

    else:
        cv2.imshow('Webcam (not cropped)', img)
        cv2.setMouseCallback("Webcam (not cropped)", click_and_crop)

cam.release()
cv2.destroyAllWindows()

This cell can be used to predict a single image, you can put the path to the image in the first line.

In [ ]:
test_image = import_test_sample("")

image1 = preprocess_image(test_image)
CNNModel1_predictions, CNNModel2_predictions = predict(image1)

predictions = [CNNModel1_predictions, CNNModel2_predictions]

test_image = ImageOps.grayscale(test_image)

test_image = np.array(test_image)
test_image = test_image.astype("int32")
test_image = test_image/255.0
test_image = test_image.reshape(48, 48, 1)

predictions.append(sgdModel.predict_proba(np.reshape(test_image, (1, 2304)))[0])

y_pred = predictCombined(predictions)

clear_output(wait=True)

print("Predicted emotion:\n", CLASS_NAMES_WITHOUT_DISGUST[np.argmax(y_pred[0])])

plt.bar(CLASS_NAMES_WITHOUT_DISGUST, y_pred[0])
plt.show()